In [1]:
import json
from ckiptagger import construct_dictionary, WS, POS
ws = WS("./data")

C:\Users\Tibame_T14\anaconda3\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\Users\Tibame_T14\anaconda3\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
C:\Users\Tibame_T14\anaconda3\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:993: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(


In [2]:
with open('./youtube/醫療保健text.json') as f:
    data = json.loads(f.read())
with open('./stopword.txt', encoding='utf-8') as f:
    stop = f.read()
stopword = stop.split('\n')
# 使用分好詞的資料
alltext = []
for file in data.values():
    for name, texts in file.items():
        word_sentence_list = ws(list(texts))
        text_list = []
        for i in range(len(texts)):
            for word in word_sentence_list[i]:
                if word not in stopword:
                    text_list.append(word)
        if text_list != []:
            alltext.append(text_list)

In [3]:
from gensim.models import Word2Vec
model1 = Word2Vec(sentences=alltext, min_count=1, workers=4)

In [4]:
from gensim.corpora import Dictionary
dct = Dictionary(alltext)

In [5]:
maxlen = 0
for i in [len(k) for k in alltext]:
    if i > maxlen:
        maxlen = i

In [6]:
X = []
for document in alltext:
    doc = model1.wv[document]
    doclist = []
    for i in doc:
        num = []
        for n in i:
            num.append(n)
        doclist.append(num)
    doclen = len(doclist)
    for k in range(maxlen-doclen):
        doclist.append([len(dct) for _ in range(100)])
    X.append(doclist)

In [7]:
import random
y = [str(random.randint(0,3)) for _ in range(len(alltext))]

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.3,random_state=0)

In [11]:
from keras.models import Model
from keras.layers import *

main_input = Input(shape=(100,), dtype='float64')
# 嵌入层（使用预训练的词向量）
embedder = Embedding(len(dct) + 1, 300, input_length=100, trainable=False)
embed = embedder(main_input)
# 卷积层和池化层，设置卷积核大小分别为3,4,5
cnn1 = Conv1D(256, 3, padding='same', strides=1, activation='relu')(embed)
cnn1 = MaxPooling1D(pool_size=48)(cnn1)
cnn2 = Conv1D(256, 4, padding='same', strides=1, activation='relu')(embed)
cnn2 = MaxPooling1D(pool_size=47)(cnn2)
cnn3 = Conv1D(256, 5, padding='same', strides=1, activation='relu')(embed)
cnn3 = MaxPooling1D(pool_size=46)(cnn3)
# 合并三个模型的输出向量
cnn = concatenate([cnn1, cnn2, cnn3], axis=-1)
flat = Flatten()(cnn)
drop = Dropout(0.2)(flat) #在池化层到全连接层之前可以加上dropout防止过拟合
main_output = Dense(4, activation='softmax')(drop)
model = Model(inputs=main_input, outputs=main_output)
model.summary()
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 100, 300)     8006400     ['input_2[0][0]']                
                                                                                                  
 conv1d_3 (Conv1D)              (None, 100, 256)     230656      ['embedding_1[0][0]']            
                                                                                                  
 conv1d_4 (Conv1D)              (None, 100, 256)     307456      ['embedding_1[0][0]']            
                                                                                            

In [ ]:
model.fit(x=X_train,y=y_train, validation_split=0.1, epochs=10, verbose=1)